## 音声認識モデルとLLMの組み合わせ

このNotebookでは、OpenShift AIでServingされた音声認識モデルのFaster-Whisperを利用して音声から文章を起こし、LLMを使用して請求文章を分析し、話し手の心理状態や事故の場所と日時を特定します。

### 必要なライブラリとインポート

Labの指示に従って適切なワークベンチイメージを選択して起動した場合、必要なすべてのライブラリがすでにインストールされているはずです。もしインストールされていない場合は、次のセルの最初の行のコメントを外して正しいパッケージをすべてインストールしてください。その後、必要なライブラリをインポートします。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # Uncomment only if you have not selected the right workbench image

import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

### 音声認識モデルのAPIクライアント

音声認識モデルのAPIを呼び出すクライアントを定義します。

In [ ]:
# 音声認識モデルのAPIのURL
transcription_server_url = "_INFERENCE_URL_TRANSCRIPTION_"

from openai import OpenAI
openai_client = OpenAI(
    api_key="EMPTY",
    base_url= f"{transcription_server_url}/v1"
)

### Langchainパイプライン

Langchainを使用して、パイプラインを定義します。

In [ ]:
# LLM推論APIのURL
inference_server_url = "_INFERENCE_URL_LLM_"

# LLMの定義
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base= f"{inference_server_url}/v1",
    model_name="/mnt/models/",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

以下は、タスクに対してフォーマットされた**テンプレート**です。

In [ ]:
system_template_string = """
あなたは、親切で、礼儀正しく、正直なアシスタントです。
常に気配りと尊重をもって接し、真摯にサポートします。できる限り有用な返答を提供しますが、安全を確保します。
有害で、倫理に反する、偏見のある、または否定的な内容は避けます。返答が公正でポジティブなものであることを確認します。
"""

user_template_string = """
与えられた文章の内容をもとに、与えられた質問に答えてください。

### 文章:
{text}

### 質問:
{query}

### 回答:
"""

system_template = SystemMessagePromptTemplate.from_template(system_template_string)
user_template = HumanMessagePromptTemplate.from_template(user_template_string)

PROMPT = ChatPromptTemplate.from_messages([system_template, user_template])

モデルにクエリを投げるために使用する**会話**オブジェクトを作成します。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

モデルにクエリする準備が整いました。

`voices`フォルダーには、音声の例が保存されています。これらのファイルを読み込んで文字起こしを行い、それに対してLLMが行った分析を表示します。

In [ ]:
voices_path = 'voices'
onlyfiles = [f for f in listdir(voices_path) if isfile(join(voices_path, f))]

voices = {}

for filename in onlyfiles:
    audio_file= open(os.path.join(voices_path, filename), "rb")
    transcription = openai_client.audio.transcriptions.create(
      model="",
      file=audio_file
    )

    print(f"***************************")
    print(f"* 請求: {filename}")
    print(f"***************************")
    print("元の音声メッセージ:")
    print("-----------------")
    print(transcription.text)
    print('\n\n分析:')
    print("--------")
    text_input = transcription.text
    sentiment_query = "この請求の文章から読み取れる感情はどのようなものですか？「肯定的」、「否定的」、「どちらでもない」から1つだけ選んで答え、その理由もあわせて説明してください。"
    location_query = "この請求に関連する出来事はどこで起こりましたか？出来事の発生した場所について、市区町村や通りの名前などを答えて下さい。"
    time_query = "この請求に関連する出来事はいつ起こりましたか？日付と、時刻あるいは時間帯を一言で答えて下さい。"
    print(f"- 送信者の感情: ")
    conversation.predict(text=text_input, query=sentiment_query);
    print("\n- 発生場所: ")
    conversation.predict(text=text_input, query=location_query);
    print("\n- 発生日時: ")
    conversation.predict(text=text_input, query=time_query);
    print("\n\n                          ----====----\n")
    